## Для загрузки датасетов из kaggle по API необходимо положить файл kaggle.json в каталог пользователя
команды ниже копируют json в каталог .kaggle, присваивают доступ на чтение и запись владельцу файла

In [9]:
!mkdir -p ~/.kaggle
!cp ~/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## Импорт библиотек и скачивание архива с датасетом

In [8]:
import kaggle.cli
import sys
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
from datetime import date, datetime

dataset = "vivek468/superstore-dataset-final"
sys.argv = [sys.argv[0]] + f"datasets download {dataset}".split(" ")
kaggle.cli.main()

zfile = ZipFile(f"{dataset.split('/')[1]}.zip")
zfile.extractall() 
zfile.close()

data = pd.read_csv('superstore-dataset-final.zip', encoding_errors='ignore')

100%|██████████| 550k/550k [00:01<00:00, 533kB/s]


## Приводим поля с датами к типу date
Отобразим несколько строк датасета и его описание

In [10]:
data.columns = map(str.lower, data.columns)
data.columns = data.columns.str.replace(' ', '_')

data['order_date'] = data['order_date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").date()).astype('datetime64[ns]')
data['ship_date'] = data['ship_date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").date()).astype('datetime64[ns]')

display(data.head(3))
data.describe()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


,row_id,postal_code,sales,quantity,discount,profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


## Проставим широту и долготу, зная город и страну

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
# location = geolocator.geocode('Los Angeles, United States')
# print(location.address, location.latitude, location.longitude)
data['latitude'] = data.apply(lambda x: geolocator.geocode(x['city'], x['country']).latitude, axis=1)
data['longitude'] = data.apply(lambda x: geolocator.geocode(x['city'], x['country']).longitude, axis=1)


/usr/local/lib/python3.8/dist-packages/geopy/geocoders/osm.py:132: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  warnings.warn(


## Сохраним обработанные данные

In [ ]:
display(data.head(3))
data.to_csv('superstore-dataset-processed.csv') 